<h1 bold>Toronto Neighborhood Project</h1>
This model analyzes neighborhoods in Toronto and clusters neighborhoods together based on types of venues and attractions around those neighborhoods that are within reasonable walking distance.

In [1]:
# Importing necessary libraries

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1


<h2 bold> Webscrape for neighborhoods from Wiki entry</h2>
Using pandas to scrape website with details of postal codes in the toronto area, and create a dataframe of the table contents

In [2]:
# define url of Toronto Wiki entry
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695'

# Read the table from the url into a pandas dataframe
table = pd.read_html(url)
df = table[0]

<h2 bold>Review the contents of the dataframe and clean up the data </h2>

In [319]:
# Review dataframe
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [320]:
df.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M9V,Not assigned,Not assigned
freq,8,77,78


In [384]:
# remove "Not Assigned" values, remove on PO Box for business related mail processing
final_df = df[df.Borough != "Not assigned"]
final_df.head()
final_df = final_df.set_index('Neighbourhood')
final_df = final_df.drop('Business Reply Mail Processing Centre 969 Eastern')
final_df = final_df.reset_index()
final_df.head()

,Neighbourhood,Postcode,Borough
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,Harbourfront,M5A,Downtown Toronto
3,Regent Park,M5A,Downtown Toronto
4,Lawrence Heights,M6A,North York


In [385]:
# review final dataframe
final_df.describe()

,Neighbourhood,Postcode,Borough
count,210,210,210
unique,208,102,11
top,Runnymede,M9V,Etobicoke
freq,2,8,45


In [386]:
final_df.shape

(210, 3)

Add the Latitude and Longitude for each postal code to the dataframe

In [359]:
# install libraries for geocoder
!conda install -c conda-forge geocoder -y
import geocoder # import geocoder 

   

Solving environment: done

# All requested packages already installed.



Import csv file of post codes and lat/long (this cell was removed prior to saving notebook to protect keys and API credentials)

*******HIDDEN CELLS*******

In [387]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_4100a76846bb4eb681808ba6b9760a11 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_4100a76846bb4eb681808ba6b9760a11 = 'https://s3.us.cloud-object-storage.appdomain.cloud'

client_4100a76846bb4eb681808ba6b9760a11 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AGKi8r1mHQpZRABv8FFVxvh3CBOep79vrMnIAW4hjJ7-',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_4100a76846bb4eb681808ba6b9760a11)

body = client_4100a76846bb4eb681808ba6b9760a11.get_object(Bucket='datasciencecapstone-donotdelete-pr-zrsybhtyxo1pqn',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.columns=['Postcode', 'Lat', 'Lng']


Review uploaded csv as dataframe

In [388]:
df_data_1.head()

,Postcode,Lat,Lng
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging 2 dataframes together to get final df

In [389]:
result = pd.merge(df_data_1, final_df)
result.head()

,Postcode,Lat,Lng,Neighbourhood,Borough
0,M1B,43.806686,-79.194353,Rouge,Scarborough
1,M1B,43.806686,-79.194353,Malvern,Scarborough
2,M1C,43.784535,-79.160497,Highland Creek,Scarborough
3,M1C,43.784535,-79.160497,Rouge Hill,Scarborough
4,M1C,43.784535,-79.160497,Port Union,Scarborough


Filter for all neighborhoods in Toronto Only

In [424]:
toronto_df = result[result['Borough'].str.contains('Toronto')]



<h2 bold>Create a map of Toronto and neighborhoods </h>

In [391]:
#center map around lat/long of Toronto
address = 'Toronto, CA'

# initiate a geolocator
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude])

# add markers to map based on neighborhood data
for lat, lng, borough, neighborhood in zip(toronto_df['Lat'], toronto_df['Lng'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

<h2 bold>Explore the neighborhoods and create a new dataframe to store venue information by neighborhood </h1>

<div>Hidden Cell with 4Square API credentials</div>
This cell was removed prior to saving the document to protect API keys and credentials

*******HIDDEN CELLS********

In [392]:
#define 4Square API credentials
CLIENT_ID = 'BGQPGOQZSSINTFY3JJ0PQEN2SPQEWUZHUWSOXVABMGF3X4TL' # your Foursquare ID
CLIENT_SECRET = 'XUWVWLUHTHHIUQDM0AWPURDOGT2REBJZXUYGLZ0WTX1LNBDM' # your Foursquare Secret
ACCESS_TOKEN = 'ZPY4KYWUFRBJ4ERYORQLLOGE14HHUYJTN4SBX3AZBYBTSOR3' # your FourSquare Access Token
VERSION = '20210606'
LIMIT = 100


In [393]:
#define a function to create a "venues_list" dataframe for all neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [394]:
# run the function on the toronto neighborhood locations, storing the venue information in the "toronto_venues" dataframe

toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Lat'],
                                   longitudes=toronto_df['Lng']
                                  )

<div>Explore toronto_venues dataframe to understand logic</div>


In [395]:
toronto_venues.shape


(2912, 7)

In [396]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Beaches Dance & Music Studio,43.680595,-79.291300,Dance Studio
4,The Danforth West,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [397]:
# see how many venues were returned in each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,16,16,16,16,16,16
Berczy Park,46,46,46,46,46,46
Brockton,22,22,22,22,22,22
CN Tower,16,16,16,16,16,16
Cabbagetown,38,38,38,38,38,38
Central Bay Street,61,61,61,61,61,61
Chinatown,60,60,60,60,60,60
Christie,14,14,14,14,14,14


In [398]:
#show how many unique categories there are for the venues listed
len(toronto_venues['Venue Category'].unique())

211

<h2 bold>Summarize each neighborhood based on venue frequency, category</h2>

In [399]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# # move neighborhood column to the first column
# fixed_columns = [toronto_onehot.Neighborhood + list(toronto_onehot.columns[:-1])
# toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Group each category by neighborhood, and the frequency of each category in each neighborhood

In [400]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Adelaide,0.0,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.01,0.0,0.0,0.010000,0.0,0.02,0.0,0.0,0.010000,0.010000,0.02,0.010000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.01,0.010000,0.0,0.0,0.01,0.02,0.0,0.000000,0.060000,0.0,0.0,0.0,0.000000,0.01,0.01,0.0,0.000000,0.04,0.000000,0.170000,0.0,0.0,0.0,0.01,0.000000,0.0,0.02,0.000000,0.02,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.000000,0.000000,0.040000,0.01,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.01,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.010000,0.0,0.0,0.01,0.0,0.010000,0.000000,0.0,0.0,0.0,0.02,0.0000,0.0,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0000,0.030000,0.0,0.0,0.01,0.050000,0.0,0.0000,0.010000,0.0,0.

### Create a new dataframe with each neighborhood listed with their top 5 venues

In [401]:
# Define a new function to find the top venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [402]:
# execute the function and create a new dataframe with the neighborhoods listed with their top 5 venue categories
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tor_venues = pd.DataFrame(columns=columns)
tor_venues['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    tor_venues.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

tor_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adelaide,Coffee Shop,Café,Sandwich Place,Gym,Clothing Store
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane
2,Berczy Park,Cocktail Bar,Coffee Shop,Sandwich Place,Bakery,Seafood Restaurant
3,Brockton,Café,Breakfast Spot,Coffee Shop,Sandwich Place,Furniture / Home Store
4,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane


## Let's cluster the neighborhoods together using KMeans Clustering

In [403]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 1, 3, 1, 1, 1, 1, 1], dtype=int32)

In [404]:
# add clustering labels
tor_venues.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(tor_venues.set_index('Neighborhood'), on='Neighbourhood')


### Now we can visulaize the clusters to see the representation on a map

In [405]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Lat'], toronto_merged['Lng'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now we can examine each cluster for commonality and name each appropriately

<h5>Cluster0 (Red): Restaurants & Playgrounds (2 neighborhoods)</h5>
<h5>Cluster1 (Purple): Coffee Shops, Cafes. Restuarants(59 neighborhoods) <h/5>
<h5>Cluster2 (Blue): Gardens (1 neighborhood)<h/5>
<h5>Cluster3 (Green): Airport & Railways (8 neighborhoods)</h5>
    <h5>Cluster4 (Orange): Parks & Trails (4 neighborhoods)</h5>    
    

In [412]:
cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster0.head()

,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
80,43.689574,0,Restaurant,Playground,Tennis Court,Adult Boutique,Museum
81,43.689574,0,Restaurant,Playground,Tennis Court,Adult Boutique,Museum


In [415]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster1.head()


,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
67,43.676357,1,Health Food Store,Pub,Dance Studio,Music Venue,Men's Store
71,43.679557,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Yoga Studio
72,43.679557,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Yoga Studio
73,43.668999,1,Fast Food Restaurant,Park,Pizza Place,Steakhouse,Sandwich Place
74,43.668999,1,Fast Food Restaurant,Park,Pizza Place,Steakhouse,Sandwich Place


In [417]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster2.head()

,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
110,43.711695,2,Garden,Malay Restaurant,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [423]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster3.head()

,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
121,43.628947,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane
122,43.628947,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane
123,43.628947,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane
124,43.628947,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane
125,43.628947,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane


In [420]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster4.head()


,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
76,43.728020,4,Park,Swim School,Bus Line,Adult Boutique,Movie Theater
87,43.679563,4,Park,Playground,Trail,Adult Boutique,Movie Theater
111,43.696948,4,Park,Trail,Sushi Restaurant,Jewelry Store,Adult Boutique
112,43.696948,4,Park,Trail,Sushi Restaurant,Jewelry Store,Adult Boutique
